In [ ]:
#Need to create list of synonyms and antonyms from words found on Thesaurus.com and Word.net
#First thought is to create a list of words of interest that can be queried directly by our script 
#Need to also create a list of list of the words that we used to create the syn-ant query list.

In [1]:
#For Thesaurus.com (T.com)
import numpy as np
import pandas as pd
import selenium
from selenium import webdriver
import bs4
from bs4 import BeautifulSoup as bs
import time
import re
import nltk
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from collections import defaultdict
import requests
import time


In [ ]:
#Pulling "useful_words" from the MacMillan Dictonary

useful_words_MacMillan = []

driver = webdriver.Chrome()
driver.get('https://www.macmillandictionary.com/us/thesaurus-category/american/describing-chemicals-and-chemical-processes')
time.sleep(2)

html_doc = driver.page_source # stores the source HTML code in the driver's page_source attribute
   
#words = re.findall(r'"<h3>"(\S*?)"</h3>"', html_doc) 

soup = bs(html_doc)

for a in soup.find_all('h3'):
    useful_words_MacMillan.append(a.string)
    
driver.quit()



In [ ]:
sample_set = ['corrosion', 'flammable', 'resistant', 'retardate', 'malleable', 'flexible', 'active', 'cm', 'mm',
              'nm', 'angstrom','inhibitor', 'anticorrosive', 'impedance', 'steel', 'concentration', 'adsorption',
              'absorption', 'surface', 'fe', 'polarization', 'percentage', 'studies', 'measurements', 'immersion',
              'loss', 'acid', 'carbon', 'decontamination', 'efficiency', 'electrochemical', 'water', 'coatings',
              'protection', 'penetration', 'al', 'oxidation', 'copper', 'metal', 'mol', 'rate', 'ph', 'brass',
              'cathodic', 'environment', 'performance', 'elastic', 'constant', 'refractive', 'inductive', 'zinc',
              'external', 'sodium', 'free', 'thermal', 'decomposition', 'organic', 'exposure', 'crystalline',
              'protective', 'pb', 'lead', 'alkaline', 'asymmetric', 'additive', 'homogeneous', 'volatile',
              'stainless', 'soil', 'reduction', 'sour', 'layer', 'negative', 'durable', 'autonomous', 'hazardous',
              'absorb', 'similar', 'pattern', 'physical', 'activation', 'excess', 'transparent', 'synthesis',
              'reactive', 'uniform', 'cohesive', 'conductive', 'hindrance', 'feedback', 'gaseous', 'barrier',
              'aggregation', 'inclusion', 'microscopic', 'ethane', 'sulfur', 'cracking', 'mineral', 'stability', 'magnesium']
sample_syn_list = []
sample_ant_list = []

driver = webdriver.Chrome()

for word in sample_set:
    
    driver.get('view-source:https://www.thesaurus.com/browse/' + word)
    time.sleep(2)
    
    html_doc = driver.page_source # stores the source HTML code in the driver's page_source attribute
   
    #Issue with next two lines is that we are also picking up "Related Terms" synonyms and antonyms.
    #Dirty fix is to keep only the 5 ant and syn results from a query after it has been cleaned.
    syn = re.findall(r'"similarity":"100","isInformal":"0","isVulgar":null,"term":"(\S*?)","targetTerm"', html_doc) 
    ant = re.findall(r'"similarity":"-100","isInformal":"0","isVulgar":null,"term":"(\S*?)","targetTerm"', html_doc)
    
    sample_syn_list.append(syn)
    sample_ant_list.append(ant)
    
    driver.refresh()

driver.quit()


    

In [ ]:
#DataFrame creation based off amount of 'useful_words'
sample_array = np.empty((len(sample_set),3))
sample_array_df = pd.DataFrame(data=sample_array[0:,:], columns = {'Term', 'Synonyms', 'Antonyms'})

#Creation of raw list of lists of syn/ant for each 'useful_word'
raw_syn_list = [[] for i in range(len(sample_set))]
raw_ant_list = [[] for i in range(len(sample_set))]


In [ ]:
#Clean up scraped data (remove repetitions, need to find out how many syn/ant we want to keep)
#Populate raw list of lists with cleaned syn/ant for each'useful_word'
for i in range(len(sample_set)):
    for word in sample_syn_list[i]:
        if word in raw_syn_list[i]:
            continue
        else:
            raw_syn_list[i].append(word)
    
for i in range(len(sample_set)):
    for word in sample_ant_list[i]:
        if word in raw_ant_list[i]:
            continue
        else:
            raw_ant_list[i].append(word)

In [1]:
#Populate DataFrame with 'useful_word' and syn/ant for each 'useful_word'
#For each term we have a list of strings for its 'Antonyms' and 'Synonyms'
for i in range(len(sample_array_df)):
    sample_array_df['Term'].loc[i] = sample_set[i]
    sample_array_df['Synonyms'].loc[i] = raw_syn_list[i][:5]
    sample_array_df['Antonyms'].loc[i] = raw_ant_list[i][:5]
sample_array_df

NameError: name 'sample_array_df' is not defined

In [ ]:
data = '/Users/Thomas/Desktop/BETO2020-Local/Ant_Syn_Scraping'
os.chdir(data)
sample_T_com_df = pd.read_csv('T.com_SynAntList.csv', index_col=0)

In [ ]:
def remove_apostrophe(word):
    word = word[1:-1]
    return word

In [ ]:
def clean_dataframe(df):
    for i in range(0,len(df)):
        df.iloc[i,1] = df.iloc[i,1][1:-1] #Getting rid of the brackets around the tokens in Tokens column
        df.iloc[i,2] = df.iloc[i,2][1:-1]
    
    for i in range(0,len(df)):
        df.iloc[i,1] = df.iloc[i,1].split(', ') #Seperating each pair word with a ', '
        df.iloc[i,2] = df.iloc[i,2].split(', ')
    
    for i in range(0,len(df)):
        df.iloc[i,1] = list(map(remove_apostrophe, df.iloc[i,1])) #removine apostrophes around each word
        df.iloc[i,2] = list(map(remove_apostrophe, df.iloc[i,2]))

In [ ]:
import nltk
import re
from nltk.corpus import wordnet
sample_set = ['corrosion', 'flammable', 'resistant', 'retardate', 'malleable', 'flexible', 'active', 'cm', 'mm', 'nm', 'angstrom','inhibitor', 'anticorrosive', 'impedance', 'steel', 'concentration', 'adsorption', 'absorption', 'surface', 'fe', 'polarization', 'percentage', 'studies', 'measurements', 'immersion', 'loss', 'acid', 'carbon', 'decontamination', 'efficiency', 'electrochemical', 'water', 'coatings', 'protection', 'penetration', 'al', 'oxidation', 'copper', 'metal', 'mol', 'rate', 'ph', 'brass', 'cathodic', 'environment', 'performance', 'elastic', 'constant', 'refractive', 'inductive', 'zinc', 'external', 'sodium', 'free', 'thermal', 'decomposition', 'organic', 'exposure', 'crystalline', 'protective', 'pb', 'lead', 'alkaline', 'asymmetric', 'additive', 'homogeneous', 'volatile', 'stainless', 'soil', 'reduction', 'sour', 'layer', 'negative', 'durable', 'autonomous', 'hazardous', 'absorb', 'similar', 'pattern', 'physical', 'activation', 'excess', 'transparent', 'synthesis', 'reactive', 'uniform', 'cohesive', 'conductive', 'hindrance', 'feedback', 'gaseous', 'barrier', 'aggregation', 'inclusion', 'microscopic', 'ethane', 'sulfur', 'cracking', 'mineral', 'stability', 'magnesium']

WN_syn_list = []
WN_ant_list = []

for word in sample_set:
    
    synonyms = []
    antonyms = []
    
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonyms.append(l.name())
        if l.antonyms():
                antonyms.append(l.antonyms()[0].name())
    
    clean_syns = []
    clean_ants = []

    for word in range(len(set(synonyms))):
        clean_syn = (list(set(synonyms)))[word].replace("_", " ")
    
        clean_syns.append(clean_syn)

    for word in range(len(set(antonyms))):
        clean_ant = (list(set(antonyms)))[word].replace("_", " ")
    
        clean_ants.append(clean_ant)
        
    WN_syn_list.append(clean_syns)
    WN_ant_list.append(clean_ants)
    

In [ ]:
#Adding WN sourced words to the T.com df
#Appending WN_syns or WN_ants to the 
for i in range(0,len(sample_T_com_df)):
    if sample_T_com_df.iloc[i,1] == ['']:
        sample_T_com_df.iloc[i,1] = WN_ant_list[i]
    else:
        continue
    if sample_T_com_df.iloc[i,2] == ['']:
        sample_T_com_df.iloc[i,2] = WN_syn_list[i]
    else:
        continue


In [ ]:
sample_WN_Tcom_df = sample_T_com_df

In [ ]:
sample_T_com_df.head()

In [ ]:
def convert(olddf): 
    master = []

    for i in range(len(olddf)):
        term = olddf.iloc[i,0]
        antlist = olddf.iloc[i,1]
        synlist = olddf.iloc[i,2]

        if antlist is not []:
            for ant in antlist:
                master.append([term,ant,'ant'])

        if synlist is not []:
            for syn in synlist:
                master.append([term,syn,'syn'])

    return pd.DataFrame(master)

In [ ]:
path = '/Users/Thomas/Desktop/BETO2020-Local/Ant_Syn_Scraping/Sample_Refined_SynAntList.csv'
df = pd.read_csv(path, index_col = 0)

In [ ]:
convert(df)

In [ ]:
#Chemical entity normalization using pubchempy
#Can normalize by: molecular_formula, iupac_name, and synonyms
#Can even extract particular properties of the 
import pubchempy as pcp
from mendeleev import element

In [ ]:
pde_array = np.empty((110,3))
pde_df = pd.DataFrame(data=pde_array[0:,:], columns = {'IUPAC','Element', 'Molecular Formula'})

for i in range(1,110):
    compound_ID = pcp.get_cids(element(i).name, 'name', list_return='flat')
    compound = pcp.Compound.from_cid(compound_ID)
    iupac = compound.iupac_name
    formula = compound.molecular_formulaa
    pde_df['Element'].iloc[i] = element(i).name
    pde_df['IUPAC'].loc[i] = iupac
    pde_df['Molecular Formula'].loc[i] = formula

pde_df